In [130]:
import time

import numpy as np
import scipy as sc

from src.Tools import portfolio_metrics

from qiskit_src.ansatz import (QAOA, 
                               CP_QAOA)

from qiskit_src.tools import (get_qubo, 
                              min_cost_partition)


In [139]:
layers = 5
max_iter = 200
N=2
k=1
seed=2
alpha=0.01

In [140]:

expected_returns, covariances = portfolio_metrics(n=N,
                                                  seed=seed)

constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=N,k=k,mu=expected_returns,sigma=covariances,alpha=alpha)
Q, offset = get_qubo(mu=expected_returns, sigma=covariances, alpha=alpha, lmbda=lmbda, k=k)


# Initial guess for parameters (gamma, beta) of circuit
theta_min, theta_max = -np.pi, np.pi
theta_i = np.random.uniform(low=theta_min, high=theta_max, size=2*layers)
# ------ Optimizer run ------ #
_available_methods_ = ['COBYLA', 'Nelder-Mead']
_method_idx_ = 0

In [143]:
# Create an instance of QAOA
ansatz = QAOA(N_qubits=N, 
              layers=layers, 
              QUBO_matrix=Q, 
              QUBO_offset=offset)

# Initial guess for parameters (gamma, beta) of circuit
theta_min, theta_max = -np.pi, np.pi
theta_i = np.random.uniform(low=theta_min, high=theta_max, size=2*layers)  # Adjust size for gamma and beta

start_time = time.time()

# Use the get_cost method of the specific ansatz instance
res = sc.optimize.minimize(fun=ansatz.get_cost, x0=theta_i,
                           method=_available_methods_[_method_idx_],
                           options={'disp': False, 'maxiter': max_iter})

end_time = time.time()
print(f'ms: {(end_time-start_time)*1000}')
res

ms: 615.1649951934814


 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -0.06878302130789307
       x: [ 2.283e+00  9.191e-01 -1.375e+00  1.472e+00  2.613e+00
           -1.006e+00  3.969e+00 -2.097e+00 -2.478e-01 -1.339e+00]
    nfev: 200
   maxcv: 0.0

In [144]:
# Create an instance of QAOA
ansatz = CP_QAOA(N_qubits=N, 
                 cardinality=k, 
                 layers=layers, 
                 QUBO_matrix=Q, 
                 QUBO_offset=offset,
                 with_z_phase=True)

# Initial guess for parameters (gamma, beta) of circuit
theta_min, theta_max = -np.pi, np.pi
N_xx_yy_angles = layers*(N-1) 
N_z_angles = layers*N 
theta_i = np.random.uniform(low=theta_min, high=theta_max, size=N_xx_yy_angles + N_z_angles) 

start_time = time.time()

# Use the get_cost method of the specific ansatz instance
res = sc.optimize.minimize(fun=ansatz.get_cost, x0=theta_i,
                           method=_available_methods_[_method_idx_],
                           options={'disp': False, 'maxiter': max_iter})

end_time = time.time()
print(f'ms: {(end_time-start_time)*1000}')
res

ms: 1142.2100067138672


 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.22116695456816665
       x: [ 6.854e-01  2.778e+00 ...  2.962e+00  1.858e+00]
    nfev: 173
   maxcv: 0.0

In [145]:
dictt = ansatz.get_state_probabilities(angles=res.x, flip_states=False)

In [146]:
res = ''
for s in constrained_result['s']:
    res += str(s)
res

'10'

In [147]:
dictt[res]

0.999999932866877

In [148]:
import os
from concurrent.futures import ProcessPoolExecutor
import sys

from qiskit.exceptions import QiskitError
from qiskit.utils.multiprocessing import local_hardware_info
from qiskit.tools.events.pubsub import Publisher
from qiskit import user_config


def get_platform_parallel_default():
    """
    Returns the default parallelism flag value for the current platform.

    Returns:
        parallel_default: The default parallelism flag value for the
        current platform.

    """
    # Default False on Windows
    if sys.platform == "win32":
        parallel_default = False
    # On macOS default false on Python >=3.8
    elif sys.platform == "darwin":
        parallel_default = False
    # On linux (and other OSes) default to True
    else:
        parallel_default = True

    return parallel_default


CONFIG = user_config.get_config()

if os.getenv("QISKIT_PARALLEL", None) is not None:
    PARALLEL_DEFAULT = os.getenv("QISKIT_PARALLEL", None).lower() == "true"
else:
    PARALLEL_DEFAULT = get_platform_parallel_default()

# Set parallel flag
if os.getenv("QISKIT_IN_PARALLEL") is None:
    os.environ["QISKIT_IN_PARALLEL"] = "FALSE"

if os.getenv("QISKIT_NUM_PROCS") is not None:
    CPU_COUNT = int(os.getenv("QISKIT_NUM_PROCS"))
else:
    CPU_COUNT = CONFIG.get("num_process", local_hardware_info()["cpus"])

In [150]:
PARALLEL_DEFAULT

False

In [151]:
sys.platform

'darwin'